Przerób skrypt z poprzedniego zadania, tak aby generował wiele PDFów na podstawie danych z pliku `sprzęt.csv`. Kolejne pliki niech nazywają się `protokol-1.pdf`, `protokol-2.pdf`, itd.

In [1]:
import pandas as pd
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
pdfmetrics.registerFont(TTFont('Calibri', '/Applications/Microsoft Word.app/Contents/Resources/DFonts/Calibri.ttf'))
from reportlab.lib.pagesizes import A2
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image,PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from datetime import datetime

In [2]:
df = pd.read_csv("sprzęt.csv")
df

,Imię,Nazwisko,Pesel,Sprzęt 1,Sprzęt 2,Sprzęt 3,Sprzęt 4
0,Artur,Nowak,89121234567,Laptop XYZ o numerze seryjnym QWE4567,Klawiatura,Mysz,Zasilacz
1,Maria,Kowalska,97111278978,Laptop ABC o numerze seryjnym 56567878,Klawiatura,Mysz,Zasilacz
2,Joanna,Wójcik,99010123123,Tablet XXY o numerze seryjnym 34234,Ładowarka,Samochód służbowy o numerze rejestracyjnym WA9...,NaN


In [3]:
equipment_list = [x for x in df.columns if 'Sprzęt' in x]
equipment_list

['Sprzęt 1', 'Sprzęt 2', 'Sprzęt 3', 'Sprzęt 4']

In [4]:
CITY = 'Warszawa'
COMPANY_NAME = 'Przykładowa nazwa firmy'
COMPANY_ADDRESS = 'Adres firmy'
COMPANY_NIP = '9898767654'
COMPANY_REGON = '565434321'

In [5]:
def create_report(index,data,equipment_list,city, company_name,company_address,company_nip,company_regon,\
                  employee_name,employee_pesel):
    file_name = f'protokol-{index+1}.pdf'
    doc = SimpleDocTemplate(file_name,pagesize=A2,rightMargin=72,
                            leftMargin=72,topMargin=72,bottomMargin=18)
    story=[]
    styles=getSampleStyleSheet()
    date = datetime.now().strftime("%m/%d/%Y")
    form_style = ParagraphStyle('form',
                           fontName="Calibri",
                           fontSize=16,
                           parent=styles['Normal'],leftIndent=36,
                          leading=40)

    ptext = f"<para alignment='center'>{city} {date}</para>"
    story.append(Paragraph(ptext, form_style))

    ptext = f"<font>Protokół przekazania sprzętu</font>"
    story.append(Paragraph(ptext, form_style))
    story.append(Spacer(1, 50))
    story.append(Paragraph("<font>Przekazujący:</font>", form_style))
    story.append(Paragraph(f"{company_name}", form_style))
    story.append(Paragraph(f"{company_address}", form_style))
    story.append(Paragraph(f"NIP: {company_nip}",form_style))
    story.append(Paragraph(f"REGON: {company_regon}", form_style))
    story.append(Spacer(1, 50))
    story.append(Paragraph("<font>Odbierający:</font>", form_style))
    story.append(Paragraph(f"{employee_name}", form_style))
    story.append(Paragraph(f"PESEL: {employee_pesel}", form_style))
    story.append(Spacer(1, 50))
    story.append(Paragraph(f"W dniu {date} zostały przekazane następujące sprzęty:", form_style))

    for x in equipment_list:
        if pd.notnull(data[x]):
            story.append(Paragraph(f"- {data[x]}", form_style))

    story.append(Spacer(1, 50))
    story.append(Paragraph("................................", form_style))
    story.append(Paragraph("Data i podpis przekazującego", form_style))
    story.append(Paragraph("................................", form_style))
    story.append(Paragraph(f"Data i podpis odbierającego", form_style))

    doc.build(story)

In [6]:
for index, row in df.iterrows():
    employee_name = row['Imię'] + " " + row['Nazwisko']
    employee_pesel = row['Pesel']
    create_report(index, row,equipment_list,CITY, COMPANY_NAME,COMPANY_ADDRESS,COMPANY_NIP,COMPANY_REGON,employee_name,employee_pesel)